#  MITRAL STENOSIS CLASSIFICATION USING DEEP LEARNING

## Importing Neccessary Libraries

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
import os
import h5py
import cv2
import glob
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
%matplotlib inline

In [ ]:
import os
normal = os.listdir('./test/normal/')
print(normal[:10]) #the output we get are the .jpg files

stenosis = os.listdir('./test/stenosis/')
print('\n')
print(stenosis[:10])

## Visualization of Infected and Uninfected Cells

1. Infected Data

In [ ]:

import cv2
plt.figure(figsize = (12,12))
for i in range(4):
    plt.subplot(1, 4, i+1)
    img = cv2.imread('./test/normal' + "/" + normal[i])
    plt.imshow(img)
    plt.title('stenosis')
    plt.tight_layout()
plt.show()

2. Unifected Data

In [ ]:
plt.figure(figsize = (12,12))
for i in range(4):
    plt.subplot(1, 4, i+1)
    img = cv2.imread('./test/stenosis' + "/" + stenosis[i])
    plt.imshow(img)
    plt.title('normal')
    plt.tight_layout()
plt.show()

In [ ]:
def loading_data(data_path, target_size=(512,512)):
    img_arr_img = []
    filelist_img = glob.glob(data_path+"*.jpg")
    image_paths = sorted(filelist_img)
    
    for image_path in image_paths:
        try:
            image = cv2.imread(image_path)
            image = cv2.resize(image, dsize=target_size)
            cv2.normalize(image, image, 0, 255, cv2.NORM_MINMAX)
            img_arr_img.append(image)
        except (RuntimeError, TypeError,NameError) as e:
            print(e)
            pass
    return np.asarray(img_arr_img), image_paths 

In [ ]:
training_path_stenosis = './test/stenosis/'
training_path_normal = './test/normal/'
test_path_stenosis = './test2/stenosis/'
test_path_normal = './test2/normal/'
X_stenosis, filenames_stenosis = loading_data(training_path_stenosis)
X_normal, filenames_normal = loading_data(training_path_normal)
X_stenosis1, filenames_stenosis1 = loading_data(test_path_stenosis)
X_normal1, filenames_normal1 = loading_data(test_path_normal)
print(X_stenosis.shape) 
print(X_normal.shape)
print(X_stenosis1.shape)
print(X_normal1.shape)


In [ ]:
X = np.vstack((X_normal, X_stenosis))
y = np.vstack((X_normal1, X_stenosis1))
labels = [0]*X_normal.shape[0] + [1]*X_stenosis.shape[0]
labels1 = [0]*X_normal1.shape[0] + [1]*X_stenosis1.shape[0]

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
from keras_applications.imagenet_utils import _obtain_input_shape
from keras import backend as K
from keras.layers import Input, Convolution2D, \
    GlobalAveragePooling2D, Dense, BatchNormalization, Activation,Dropout
from keras.models import Model
import keras
from keras.engine.topology import get_source_inputs
from depthwise_conv2d import DepthwiseConvolution2D #debug
#from depthwise_conv2d import DepthwiseConvolution2D #release
from keras.utils import plot_model
from keras import regularizers
'''Google MobileNet model for Keras.
# Reference:
- [MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications](https://arxiv.org/pdf/1704.04861.pdf)
'''
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.contrib.metrics.streaming_auc(y_pred, y_true)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

def MobileNet(input_tensor=None, input_shape=(256,256,3), alpha=1, shallow=True, classes=1):
    """Instantiates the MobileNet.Network has two hyper-parameters
        which are the width of network (controlled by alpha)
        and input size.
        
        # Arguments
            input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
                to use as image input for the model.
            input_shape: optional shape tuple, only to be specified
                if `include_top` is False (otherwise the input shape
                has to be `(224, 224, 3)` (with `channels_last` data format)
                or `(3, 224, 244)` (with `channels_first` data format).
                It should have exactly 3 inputs channels,
                and width and height should be no smaller than 96.
                E.g. `(200, 200, 3)` would be one valid value.
            alpha: optional parameter of the network to change the 
                width of model.
            shallow: optional parameter for making network smaller.
            classes: optional number of classes to classify images
                into.
        # Returns
            A Keras model instance.

        """

    input_shape = _obtain_input_shape(input_shape,
                                      default_size=256,
                                      min_size=96,
                                      data_format=K.image_data_format(),
                                      require_flatten=True)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    x = Convolution2D(int(32 * alpha), (3, 3), strides=(2, 2), padding='same', use_bias=False)(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = DepthwiseConvolution2D(int(32 * alpha), (3, 3), strides=(1, 1), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Convolution2D(int(64 * alpha), (1, 1), strides=(1, 1), padding='same', use_bias=False,kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-5))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = DepthwiseConvolution2D(int(64 * alpha), (3, 3), strides=(2, 2), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Convolution2D(int(128 * alpha), (1, 1), strides=(1, 1), padding='same', use_bias=False,kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-5))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = DepthwiseConvolution2D(int(128 * alpha), (3, 3), strides=(1, 1), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Convolution2D(int(128 * alpha), (1, 1), strides=(1, 1), padding='same', use_bias=False,kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-5))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = DepthwiseConvolution2D(int(128 * alpha), (3, 3), strides=(2, 2), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Convolution2D(int(256 * alpha), (1, 1), strides=(1, 1), padding='same', use_bias=False,kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-5))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = DepthwiseConvolution2D(int(256 * alpha), (3, 3), strides=(1, 1), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Convolution2D(int(256 * alpha), (1, 1), strides=(1, 1), padding='same', use_bias=False,kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-5))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)


    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    out = Dense(classes, activation='sigmoid')(x)

    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input

    model = Model(inputs, out, name='mobilenet')

    return model
model=MobileNet()
model.summary()
from keras.callbacks import ModelCheckpoint


optimizer = keras.optimizers.Adam(0.0001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])



hist = model.fit(X,labels,validation_data=(y,labels1),batch_size=2,epochs=15)



def save_history(hist, filepath):
    with open('./hhi', 'w') as f:
        json.dump(hist.history, f)

def plot_loss(history_filepath):
    with open(history_filepath) as json_data:
        history = json.load(json_data)
        #print(history)
    print(history.keys())
    plt.plot(history['loss'])
    plt.plot(history['acc'])
   #plt.plot(history['auc_roc'])
    plt.title('training results')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['loss', 'accuracy'], loc='upper left')
    plt.show()
save_history(hist,'./')
plot_loss('./hhi')
print(model.evaluate(y,labels1,batch_size=1))

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score,precision_recall_curve
from matplotlib import pyplot
from sklearn.metrics import roc_curve, auc # roc curve tools
#from sklearn.metrics import precision_recall_curve
# roc curve and auc
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

predict = model.predict(y, batch_size=1)
#print(predict)
# calculate AUC
auc = roc_auc_score(labels1, predict)
print('AUC: %.5f' % auc)
# calculate roc curve
fpr, tpr, thresholds = roc_curve(labels1, predict)
# plot no skill
pyplot.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
pyplot.plot(fpr, tpr, marker='.')
# show the plot
pyplot.show()
print("<<<<<<>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.")

In [ ]:
print(model.evaluate(y, labels1,batch_size=1))

In [ ]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(labels1, predictions)
print(average_precision)
precision, recall, _ = precision_recall_curve(labels1, predict)

plt.step(recall, precision, color='b', alpha=0.2,
         where='post')

plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))


plt.plot(recall, precision,'ro-',markevery=100) 
# naming the x axis 
plt.xlabel('x - axis') 
# naming the y axis 
plt.ylabel('y - axis') 
  
# giving a title to my graph 
plt.title('PR curveof mitral valve') 
  
# function to show the plot 
plt.show()
plt.savefig('./pr.png')
model.save('latest.h5')

In [ ]:
print(model.evaluate(y, labels1,batch_size=1))
#model.save_weights('./MVAbnor.h5')

In [ ]:
model.save('mva.h5')

##### 

In [ ]:
predictions = model.predict(y,batch_size=1)
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt

from sklearn.metrics import average_precision_score
average_precision = average_precision_score(labels1, predictions)
print(average_precision)


In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

precision, recall, _ = precision_recall_curve(labels1, predictions)

plt.step(recall, precision, color='b', alpha=0.2,
         where='post')

plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('1-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))


plt.plot(recall, precision,'ro-',markevery=100) 
# naming the x axis 
plt.xlabel('x - axis') 
# naming the y axis 
plt.ylabel('y - axis') 
  
# giving a title to my graph 
plt.title('PR curveof mitral valve') 
  
# function to show the plot 
plt.show()
plt.savefig('pr.png')


In [ ]:
model.save_weights('./mvaxxx.h5')
model.save('mva.h5')

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score,precision_recall_curve
from matplotlib import pyplot
from sklearn.metrics import roc_curve, auc # roc curve tools
#from sklearn.metrics import precision_recall_curve
# roc curve and auc
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

predict = model.predict(y, batch_size=1)
#print(predict)
# calculate AUC
auc = roc_auc_score(labels1, predict)
print('AUC: %.5f' % auc)
# calculate roc curve
fpr, tpr, thresholds = roc_curve(labels1, predict)
# plot no skill
pyplot.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
pyplot.plot(fpr, tpr, marker='.')
# show the plot
pyplot.show()
print("<<<<<<>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.")


In [ ]:
predictions = model.predict(y,batch_size=1)

fig=plt.figure(figsize=(28, 28))
columns = 2
rows = 2
random_number = np.random.randint(0,y.shape[0])
for i in range(1, columns*rows +1):
    fig.add_subplot(rows, columns, i)
    plt.imshow(y[i])
    gt = ['normal', 'abnormal']
    plt.title('abnormal likelihood {:.1%}\n Ground Truth:{} '.format(float(predictions[i]), gt[labels1[i]]))
    
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

threshold = 0.65
predictions_final = [int(pred>threshold) for pred in predictions]

print(classification_report(labels1, predictions_final))

In [ ]:
def draw_confusion_matrix(true,preds):
    conf_matx = confusion_matrix(true, preds)
    sns.heatmap(conf_matx, annot=True,annot_kws={"size": 10},fmt='g', cbar=True, cmap="viridis")
    plt.show()
    print(conf_matx)
print(draw_confusion_matrix(labels1, predictions_final))

In [ ]:
import numpy as np


def plot_confusion_matrix(cm,
                          target_names=['normal','abnormal'],
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / np.sum(cm).astype('float')
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

cm = confusion_matrix(labels1, predictions_final)
print(cm)
plot_confusion_matrix(cm)

In [ ]:
print(model.evaluate(y, labels1))
model.save_weights('cnnmva.h5') #model weights saved 